In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os

os.environ["PATH"] = "/root/.local/bin:" + os.environ["PATH"]
# Créer l'environnement
!rm -rf myenv
!uv venv /content/myenv --python 3.11

# Activer (pour les commandes shell suivantes)
os.environ["VIRTUAL_ENV"] = "/content/myenv"
os.environ["PATH"] = "/content/myenv/bin:" + os.environ["PATH"]

In [ ]:
!which python
# -> /content/myenv/bin/python

In [ ]:
!source myenv/bin/activate
%rm -rf vimseo
!git clone -b pyfr-on-colab https://github.com/SebastienBocquet/vimseo.git
%cd vimseo
!uv pip install -e . --python /content/myenv/bin/python
%cd ..

In [ ]:
!python -c "import vimseo.api"

In [ ]:
# 1. INSTALLATION DES DÉPENDANCES SYSTÈME ET PYTHON
print("--- Installation de PyFR et des dépendances ---")
!apt-get update
!apt-get install -y libopenmpi-dev openmpi-bin libmetis-dev
!uv pip install pyfr mako h5py markupsafe pyvista==0.47.0 --python /content/myenv/bin/python

print("--- Installation de gmsh et des dépendances ---")
!uv pip install gmsh --python /content/myenv/bin/python
!sudo apt install -y libglu1-mesa libgl1-mesa-glx libxrender1 libxcursor1 libxft2 libxinerama1

In [ ]:
%%writefile /content/mon_script.py
from vimseo.api import create_model
from vimseo.job_executor.pyfr_job_executor import PyFRJobSettings
from vimseo.problems.cfd.couette_2d.couette_2d import Couette2DModel

model = create_model("Couette2DModel", check_subprocess=True)
model.EXTRA_INPUT_GRAMMAR_CHECK = True
model._chain.disciplines[0].job_executor.set_options(PyFRJobSettings(backend="cuda"))
model.cache = None
model.execute()

In [ ]:
# 5. TÉLÉCHARGEMENT DU RÉSULTAT FINAL
import glob

from google.colab import files as gcfiles

input_folder = "default_scratch/Couette2DModel/Dummy/2"
file_pattern = "couette-flow-*.pyfrs"
files = sorted(glob.glob(os.path.join(input_folder, file_pattern)))
volume_file = f"{input_folder}/couette-flow.pyfrm"
!ls {volume_file}
for file in files:
    print(file)
    vtu_file = file.replace(".pyfrs", ".vtu")

    !pyfr export {volume_file} {file} {vtu_file}
    gcfiles.download(vtu_file)
    gcfiles.download(file)

In [ ]:
%cd /content/
!/content/myenv/bin/python mon_script.py